# Домашнее задание по Автобрее №2
## Настя Чижикова, БКЛ181

### Русский текст

В качестве русского текста я взяла какой-то бугурт-тред из ВКонтакте (см. файл 'russian_text.txt'): в нем много сленговых слов, но также он частеречно разнообразен. Вручную расставила части речи: для каждого слова через нижнее подчеркивание его часть речи (дом_NOUN)

In [1]:
from pymorphy2 import MorphAnalyzer
import nltk
from pymystem3 import Mystem
from tqdm import tqdm
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [2]:
with open('russian_text.txt', encoding='utf-8') as f:
    rus_text = f.read()

Сохраним в словарь

In [3]:
pos_dict = {}

for w in nltk.word_tokenize(rus_text):
    if w[0].isalpha():
        word, pos = w.split('_')
        pos_dict[word] = pos

In [4]:
len(pos_dict)

186

##### Парсинг PyMorphy

In [5]:
m = MorphAnalyzer()
pym_dict = {}

In [6]:
m.parse('как')

[Parse(word='как', tag=OpencorporaTag('CONJ'), normal_form='как', score=0.875, methods_stack=((<DictionaryAnalyzer>, 'как', 1673, 0),)),
 Parse(word='как', tag=OpencorporaTag('ADVB,Ques'), normal_form='как', score=0.09375, methods_stack=((<DictionaryAnalyzer>, 'как', 1674, 0),)),
 Parse(word='как', tag=OpencorporaTag('PRCL'), normal_form='как', score=0.03125, methods_stack=((<DictionaryAnalyzer>, 'как', 22, 0),))]

In [7]:
for w in nltk.word_tokenize(rus_text):
    word = w.split('_')[0]
    if word.isalpha():
        pos = m.parse(word)[0].tag.POS
        pym_dict[word] = pos

##### MyStem

In [8]:
m = Mystem()
mys_dict = {}

In [9]:
for w in tqdm(nltk.word_tokenize(rus_text)):  #  токенизируем везде одинаково, чтобы легче сравнить части речи
    word = w.split('_')[0]
    if word.isalpha():
        an = m.analyze(word)[0]['analysis']
        if an == []:  #  в случае, если MyStem не может определить часть речи
            pos = 'None'
        else:
            pos = an[0]['gr'].split(',')[0]
        
        mys_dict[word] = pos

100%|████████████████████████████████████████████████████████████████████████████████| 272/272 [05:44<00:00,  1.27s/it]


In [10]:
mys_dict

{'ПОПАДАЕШЬ': 'V',
 'В': 'S',
 'ЛОББИ': 'S',
 'РЕГИСТРИРУЕШЬСЯ': 'V',
 'И': 'PART=',
 'РАДОСТНО': 'ADV=',
 'СКАЧЕШЬ': 'V',
 'К': 'S',
 'ПОРТАЛУ': 'S',
 'С': 'S',
 'НАДПИСЬЮ': 'S',
 'ВЫЖИВАНИЕ': 'S',
 'ПРЕДВКУШАЯ': 'V=непрош',
 'КАК': 'ADVPRO=',
 'ВЫКОПАЕШЬ': 'V',
 'ВСЕ': 'SPRO',
 'АЛМАЗЫ': 'S',
 'МИРА': 'S',
 'УШАТАЕШЬ': 'V',
 'ВСЕХ': 'SPRO',
 'НА': 'PART=',
 'ПВП': 'None',
 'ТЕБЯ': 'SPRO',
 'ВСТРЕЧАЕТ': 'V',
 'НЕБОЛЬШОЙ': 'A=(вин',
 'СПАУН': 'S',
 'ПРЕДСТАВЛЯЮЩИЙ': 'V',
 'ИЗ': 'PR=',
 'СЕБЯ': 'SPRO=(вин|род)',
 'ПРЯМОУГОЛЬНЫЙ': 'A=(вин',
 'КАМЕННЫЙ': 'A=(вин',
 'ЗАБОР': 'S',
 'ВНУТРИ': 'ADV=',
 'КОТОРОГО': 'APRO=(вин',
 'НАТЫКАНЫ': 'V',
 'ДОРОГИ': 'S',
 'ПРИМИТИВНЫЕ': 'A=(вин',
 'ПОСТРОЙКИ': 'S',
 'ПОД': 'S',
 'ВАРПЫ': 'S',
 'НЕМНОГО': 'ADV=',
 'РАЗОЧАРОВАВШИСЬ': 'V',
 'КРЕАТИВНОСТИ': 'S',
 'МЕСТНОГО': 'A=(вин',
 'АДМИНА': 'S',
 'НАЖИМАЕШЬ': 'V',
 'КНОПКУ': 'S',
 'ОХЕРЕВАЕШЬ': 'V=непрош',
 'ОТ': 'PR=',
 'ФАКТА': 'S',
 'ЧТО': 'SPRO',
 'ТВОЙ': 'APRO=(вин',
 'ИНВЕНТАРЬ': 'S',
 'НУЛЕВОЙ':

Видим, что некоторые части речи нужно еще почистить от грамматических меток

In [11]:
for w in mys_dict:
    pos = mys_dict[w].split('=')[0]
    mys_dict[w] = pos

In [12]:
mys_dict['ДЕСЯТЬ']

'NUM'

##### Natasha

In [13]:
m = MorphVocab()
 
nat_dict = {}

In [14]:
for w in nltk.word_tokenize(rus_text):
    word = w.split('_')[0]
    if word.isalpha():
        pos = m(word)[0].pos
        nat_dict[word] = pos

Напишем словарь соответствий тегов частей речи для 2 парсеров (при разметке вручную я использовала теги PyMorphy)

In [15]:
pos_tags_mys = {'A': ['ADJF', 'ADJS'],
            'S': ['NOUN'],
            'V': ['VERB', 'PRTF', 'PRTS', 'GRND', 'INFN'],
            'ADV': ['ADVB'],
            'ANUM': ['NUM'],
            'CONJ': ['CONJ'],
            'INTJ': ['INTJ'],
            'NUM': ['NUM'],
            'PART': ['PRCL'],
            'SPRO': ['NPRO'],
            'PR': ['PREP'],
            'APRO': ['ADJF'],
            'ADVPRO': ['ADV'],
            'None': ['None']
            }

In [16]:
pos_tags_nat = {'ADJ': ['ADJF', 'ADJS'],
            'NOUN': ['NOUN'],
            'VERB': ['VERB', 'PRTF', 'PRTS', 'GRND', 'INFN'],
            'ADV': ['ADVB'],
            'NUM': ['NUM'],
            'CCONJ': ['CONJ'],
            'INTJ': ['INTJ'],
            'NUM': ['NUMR'],
            'PART': ['PRCL'],
            'PRON': ['NPRO'],
            'ADP': ['PREP'],
            'X': ['None']
            }

In [17]:
pos_tags_pym = {'ADJF': ['ADJF'],
                'ADJS': ['ADJS'],
                'NOUN': ['NOUN'],
                'VERB': ['VERB'], 
                'PRTF': ['PRTF'], 
                'PRTS': ['PRTS'], 
                'GRND': ['GRND'], 
                'INFN': ['INFN'],
                'ADVB': ['ADVB'],
                'NUM': ['NUM'],
                'CONJ': ['CONJ'],
                'INTJ': ['INTJ'],
                'NUMR': ['NUMR'],
                'PRCL': ['PRCL'],
                'NPRO': ['NPRO'],
                'PREP': ['PREP'],
                None: ['None']
                }

In [18]:
def check_accuracy(dictionary, alignment):
    correct = 0
    for word in dictionary:
        if pos_dict[word] in alignment[dictionary[word]]:
            correct += 1
        else:
            print(word, pos_dict[word], dictionary[word])  #  выводим случаи, где разметки не совпали
    return correct / len(pos_dict)

Accuracy MyStem

In [19]:
print(check_accuracy(mys_dict, pos_tags_mys))

В PREP S
И CONJ PART
К PREP S
С PREP S
КАК CONJ ADVPRO
ВСЕ ADJF SPRO
ВСЕХ NOUN SPRO
НА PREP PART
ПВП NOUN None
ВНУТРИ PREP ADV
КОТОРОГО CONJ APRO
ПОД PREP S
ЧТО CONJ SPRO
ВСЁ ADJF SPRO
ЭТО ADJF SPRO
ДЕСЯТЬ NUMR NUM
ВСЕМИ ADJF SPRO
ВО PREP PART
ВОКРУГ PREP ADV
ДОЛБАНУТОЙ ADJF V
МОДЕРН ADJF S
РАНДОМНОГО ADJF S
0.8763440860215054


Accuracy Natasha

In [20]:
print(check_accuracy(nat_dict, pos_tags_nat))

ВСЕХ NOUN ADJ
ПВП NOUN X
КОТОРОГО CONJ ADJ
ЗАБИТ PRTS ADJ
НАБИТЫЕ PRTF ADJ
ЭТО ADJF PART
СПАВНА NOUN ADJ
ДАБЫ CONJ NOUN
СВЕТЯЩЕЙСЯ PRTF ADJ
ВОКРУГ PREP ADV
ДОЛБАНУТОЙ ADJF NOUN
МОДЕРН ADJF NOUN
СКИНОВ NOUN ADJ
БАН NOUN X
РАНДОМНОГО ADJF ADV
0.9139784946236559


Accuracy PyMorphy

In [21]:
print(check_accuracy(pym_dict, pos_tags_pym))

ВСЕХ NOUN ADJF
ПВП NOUN None
КОТОРОГО CONJ ADJF
ЗАБИТ PRTS ADJS
НАБИТЫЕ PRTF ADJF
ЭТО ADJF PRCL
СПАВНА NOUN ADJS
ДАБЫ CONJ NOUN
СВЕТЯЩЕЙСЯ PRTF ADJF
ВОКРУГ PREP ADVB
ДОЛБАНУТОЙ ADJF NOUN
МОДЕРН ADJF NOUN
СКИНОВ NOUN ADJS
БАН NOUN None
РАНДОМНОГО ADJF ADVB
0.9139784946236559


### Английский текст

In [22]:
import spacy
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from flair.data import Sentence
from flair.models import SequenceTagger

In [35]:
nlp = spacy.load("en_core_web_sm")

In [72]:
with open('eng_text.txt', encoding='utf-8') as f:
    eng_text = f.read()

In [75]:
pos_eng_dict = {}  #  считываем собственную разметку

for w in nltk.word_tokenize(eng_text):
    if w[0].isalpha():
        word, pos = w.split('_')
        pos_eng_dict[word] = pos

Здесь мы делаем pos-tagging тремя парсерами по очереди и делаем для них словари

##### SpaCy

In [111]:
spc_dict = {}

tokens = nltk.word_tokenize(eng_text)
text = ''
for t in tokens:
    text += t.split('_')[0] + ' '

doc = nlp(text)
for w in doc:
    if w.text[0].isalpha():
        spc_dict[w.text] = w.pos_

##### NLTK

In [28]:
nltk_dict = {}

tokens = nltk.word_tokenize(eng_text)
text = ''
for t in tokens:
    text += t.split('_')[0] + ' '
for w in nltk.pos_tag(nltk.word_tokenize(text)):
    nltk_dict[w[0]] = w[1]

##### Flair

In [94]:
tokens = nltk.word_tokenize(eng_text)
text = ''
for t in tokens:
    if t[0].isalpha():
        text += t.split('_')[0] + ' '

sentence = Sentence(text)
tagger = SequenceTagger.load('pos')
tagger.predict(sentence)


parsed = sentence.to_tagged_string()

2020-10-17 13:15:36,165 loading file C:\Users\Анастасия\.flair\models\en-pos-ontonotes-v0.5.pt


In [96]:
fl_dict = {}

for w in parsed.split()[::2]:
    fl_dict[w] = parsed.split()[parsed.split().index(w) + 1].strip('<>')

In [97]:
parsed

'Mister <NNP> Bennet <NNP> was <VBD> so <RB> odd <JJ> a <DT> mixture <NN> of <IN> quick <JJ> parts <NNS> sarcastic <JJ> humour <NN> reserve <NN> and <CC> caprice <NN> that <IN> the <DT> experience <NN> of <IN> three <CD> and <CC> twenty <CD> years <NNS> had <VBD> been <VBN> insufficient <JJ> to <TO> make <VB> his <PRP$> wife <NN> understand <VB> his <PRP$> character <NN> Her <PRP$> mind <NN> was <VBD> less <RBR> difficult <JJ> to <TO> develop <VB> She <PRP> was <VBD> a <DT> woman <NN> of <IN> mean <JJ> understanding <VBG> little <JJ> information <NN> and <CC> uncertain <JJ> temper <NN> When <WRB> she <PRP> was <VBD> discontented <VBN> she <PRP> fancied <VBD> herself <PRP> nervous <JJ> The <DT> business <NN> of <IN> her <PRP$> life <NN> was <VBD> to <TO> get <VB> her <PRP$> daughters <NNS> married <VBN> its <PRP$> solace <NN> was <VBD> visiting <VBG> and <CC> news <NN> Bennet <NNP> was <VBD> among <IN> the <DT> earliest <JJS> of <IN> those <DT> who <WP> waited <VBD> on <IN> Bingley <NNP

В этом случае для разметки мы использовали теги Spacy, напишем словари соответствий для других теггеров (оказалось, что NLTK и FLair используют один набор тегов)

In [86]:
pos_tags_nltk = {
    'NNP': ['PROPN'],
    'NN': ['NOUN'],
    'NNS': ['NOUN'],
    'VB': ['VERB'],
    'VBD': ['VERB'],
    'VBN': ['VERB'],
    'VBG': ['VERB'],
    'VBP': ['VERB'],
    'JJ': ['ADJ'],
    'JJS': ['ADJ'],
    'JJR': ['ADJ'],
    'PRP': ['PRON'],
    'WP': ['PRON'],
    'IN': ['ADP'],
    'DT': ['DET'],
    'RB': ['ADV'],
    'RBR': ['ADV'],
    'RBS': ['ADV'],
    'WRB': ['ADV'],
    'PRP$': ['DET'],
    'MD': ['AUX'],
    'TO': ['PART'],
    'CD': ['NUM'],
    'CC': ['CCONJ']
}

In [103]:
pos_tags_spc = {
    'PROPN': ['PROPN'],
    'NOUN': ['NOUN'],
    'VERB': ['VERB'],
    'ADJ': ['ADJ'],
    'PRON': ['PRON'],
    'ADP': ['ADP'],
    'DET': ['DET'],
    'ADV': ['ADV'],
    'DET': ['DET'],
    'AUX': ['AUX'],
    'PART': ['PART'],
    'NUM': ['NUM'],
    'CCONJ': ['CCONJ'],
    'SCONJ': ['SCONJ']
}

##### Считаем Accuracy

In [88]:
def check_accuracy(dictionary, alignment):
    correct = 0
    for word in dictionary:
        if pos_eng_dict[word] in alignment[dictionary[word]]:
            correct += 1
        else:
            print(word, pos_eng_dict[word], dictionary[word])  #  выводим случаи, где разметки не совпали
    return correct / len(pos_eng_dict)

NLTK

In [89]:
print(check_accuracy(nltk_dict, pos_tags_nltk))

was AUX VBD
that SCONJ IN
had AUX VBD
been AUX VBN
to ADP TO
understanding NOUN JJ
get AUX VB
though ADV IN
should VERB MD
not PART RB
till SCONJ VB
following VERB JJ
will VERB MD
0.8773584905660378


Flair

In [98]:
print(check_accuracy(fl_dict, pos_tags_nltk))

was AUX VBD
that SCONJ IN
had AUX VBD
been AUX VBN
to ADP TO
understanding NOUN VBG
her PRON PRP$
get AUX VB
visit NOUN VB
should VERB MD
not PART RB
till SCONJ IN
will VERB MD
0.8773584905660378


SpaCy

In [114]:
print(check_accuracy(spc_dict, pos_tags_spc))

humour NOUN PROPN
understanding NOUN ADJ
0.9811320754716981


### Часть 2. Улучшение предсказаний второй домашки путем новой обработки текста

Для обработки возьмем Наташу, которая показала высокую accuracy на данных выше

Группы, которые мы хотим выделять:

1. Не + глагол в 1 лице (не рекомендую, не советую)
2. Личное местоимение/существительное в дативе + нравиться/понравиться в 3sg или 3pl (ловим сочетания "нам нравится", "мне нравятся", "ему (ребенку) понравилось")
3. Краткое причастие + наречие (ловим сочетания типа 'хорошо сделано', 'красиво оформлена' или 'плохо нарисовано')

In [152]:
nat = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()

doc = Doc('Мне понравилось, ребенку не нравится, выполнено очень хорошо')
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
display(doc.tokens)
doc.sents[0].syntax.print()

[DocToken(stop=3, text='Мне', id='1_1', head_id='1_2', rel='iobj', pos='PRON', feats=<Dat,Sing,1>),
 DocToken(start=4, stop=15, text='понравилось', id='1_2', head_id='1_0', rel='root', pos='VERB', feats=<Perf,Neut,Ind,Sing,Past,Fin,Mid>),
 DocToken(start=15, stop=16, text=',', id='1_3', head_id='1_6', rel='punct', pos='PUNCT'),
 DocToken(start=17, stop=24, text='ребенку', id='1_4', head_id='1_2', rel='iobj', pos='NOUN', feats=<Anim,Dat,Masc,Sing>),
 DocToken(start=25, stop=27, text='не', id='1_5', head_id='1_6', rel='advmod', pos='PART', feats=<Neg>),
 DocToken(start=28, stop=36, text='нравится', id='1_6', head_id='1_2', rel='conj', pos='VERB', feats=<Imp,Ind,Sing,3,Pres,Fin,Mid>),
 DocToken(start=36, stop=37, text=',', id='1_7', head_id='1_6', rel='punct', pos='PUNCT'),
 DocToken(start=38, stop=47, text='выполнено', id='1_8', head_id='1_2', rel='conj', pos='VERB', feats=<Perf,Neut,Sing,Past,Short,Part,Pass>),
 DocToken(start=48, stop=53, text='очень', id='1_9', head_id='1_10', rel='ad

        ┌► Мне         iobj
┌─┌───┌─└─ понравилось 
│ │ ┌►│    ,           punct
│ │ │ └──► ребенку     iobj
│ │ │   ┌► не          advmod
│ └►└─┌─└─ нравится    conj
│     └──► ,           punct
└────►┌─── выполнено   conj
      │ ┌► очень       advmod
      └►└─ хорошо      advmod


Функция для поиска 1 группы в тексте

In [153]:
def find_gr_one(doc):
    groups = []
    for d in doc.tokens:
        if d.text == 'не':
            head = d.head_id
            for h in doc.tokens:
                if h.id == head and h.feats['Number'] == 'Sing':
                    groups.append(d.text + ' ' + h.text)
    return groups

Для поиска 2 группы (берем только группу без "не")

In [154]:
def find_gr_two(doc):
    groups = []
    for d in doc.tokens:
        if d.pos == 'VERB' and nat(d.text)[0].normal == 'нравиться' or nat(d.text)[0].normal == 'понравиться':
            head = d.id
            is_neg = 0
            for h in doc.tokens:
                if h.head_id == head and h.text == 'не':
                    is_neg = 1
                if h.head_id == head and 'Case' in h.feats and h.feats['Case'] == 'Dat':
                    group = h.text + ' ' + d.text
            if not is_neg:
                groups.append(group)
    return groups

Функция для поиска 3 группы в тексте

In [155]:
def find_gr_three(doc):
    groups = []
    for d in doc.tokens:
        if d.pos == 'VERB' and d.feats['VerbForm'] == 'Part' and d.feats['Variant'] == 'Short':
            head = d.id
            for h in doc.tokens:
                if h.head_id == head and h.pos == 'ADV':
                    groups.append(d.text + ' ' + h.text)
    return groups

In [156]:
text = 'Мне понравилось, ребенку не нравится, выполнено очень хорошо'


doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
    
print(find_gr_one(doc))
print(find_gr_two(doc))
print(find_gr_three(doc))

['не нравится']
['ребенку понравилось']
['выполнено хорошо']


Все это можно использовать при предсказании (хотя видим, что синтаксический парсинг не идеальный - Наташа выделяет "ребенку" как зависимое от слова "понравилось")